In [14]:
import os
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Paths for datasets
datasets = ["Dataset1", "Dataset2", "Dataset3"]
output_dir = "output_Dinov2"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Pre-trained model and processor
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-11): 12 x Dinov2Layer(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2SdpaAttention(
          (attention): Dinov2SdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_pa

In [15]:
def extract_features(image_path):
    """Extract features from an image using the pre-trained model."""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [16]:
def process_dataset(dataset_name):
    """Process a single dataset: extract features, train SVM, and save results."""
    dataset_path = dataset_name
    output_path = os.path.join(output_dir, f"{dataset_name}_features.npy")

    # Check if features already exist
    if not os.path.exists(output_path):
        print(f"Extracting features for {dataset_name}...")
        features = []
        labels = []
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(os.listdir(dataset_path))}

        for class_name, class_idx in class_to_idx.items():
            class_path = os.path.join(dataset_path, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    feature_vector = extract_features(img_path)
                    features.append(feature_vector)
                    labels.append(class_idx)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

        features = np.array(features)
        labels = np.array(labels)
        np.save(output_path, {"features": features, "labels": labels})
        print(f"Features saved for {dataset_name} to {output_path}")
    else:
        print(f"Features file already exists for {dataset_name}. Loading...")

    # Load features and labels
    data = np.load(output_path, allow_pickle=True).item()
    features, labels = data["features"], data["labels"]

    # Shuffle the dataset
    indices = np.arange(len(labels))
    np.random.shuffle(indices)
    features = features[indices]
    labels = labels[indices]

    # Train an SVM classifier on the entire dataset
    svm_clf = SVC(kernel='linear', C=1.0, random_state=42)
    svm_clf.fit(features, labels)

    # Predictions using the same data
    predictions = svm_clf.predict(features)

    # Evaluate the classifier
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy for {dataset_name}: {accuracy:.2f}")

    # Detailed classification report
    print(f"\nClassification Report for {dataset_name}:")
    print(classification_report(labels, predictions, target_names=os.listdir(dataset_path)))

    # Generate confusion matrix
    conf_matrix = confusion_matrix(labels, predictions)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=os.listdir(dataset_path), yticklabels=os.listdir(dataset_path))
    plt.title(f"Confusion Matrix for {dataset_name}")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    confusion_matrix_path = os.path.join(output_dir, f"{dataset_name}_confusion_matrix.png")
    plt.savefig(confusion_matrix_path)
    plt.close()

    print(f"Confusion matrix for {dataset_name} saved to {confusion_matrix_path}")

In [17]:
# Process all datasets
for dataset in datasets:
    process_dataset(dataset)

Extracting features for Dataset1...
Features saved for Dataset1 to output_Dinov2\Dataset1_features.npy
Accuracy for Dataset1: 1.00

Classification Report for Dataset1:
              precision    recall  f1-score   support

     Laptops       1.00      1.00      1.00        20
 Smartphones       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Confusion matrix for Dataset1 saved to output_Dinov2\Dataset1_confusion_matrix.png
Extracting features for Dataset2...
Features saved for Dataset2 to output_Dinov2\Dataset2_features.npy
Accuracy for Dataset2: 1.00

Classification Report for Dataset2:
              precision    recall  f1-score   support

        Cars       1.00      1.00      1.00        20
      Planes       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00   